In [1]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import os
from IPython.display import Video


# Create output folder
os.makedirs("output", exist_ok=True)

model = YOLO("C:\\Users\\ASUS\\Downloads\\Saini Project Internshala\\a2\\best.pt")  # Must be in current dir



In [2]:
# Cell 3: Helper Functions

def extract_color_hist(img, bbox):
    x1, y1, x2, y2 = [int(i) for i in bbox]
    roi = img[y1:y2, x1:x2]
    if roi.size == 0:
        return np.zeros((256,))
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def compute_similarity(hist1, hist2):
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

def draw_tracks(frame, tracks):
    for bbox, player_id in tracks:
        x1, y1, x2, y2 = [int(i) for i in bbox]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(frame, f'ID {player_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    return frame


In [3]:
# Cell 4: Initialize Variables

input_path = "video\\15sec_input_720p.mp4"
output_path = "output/reid_output1.mp4"

cap = cv2.VideoCapture(input_path)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Core ID mapping structures
player_db = {}         # id -> hist
assigned_tracks = {}   # bbox -> id
next_id = 0

frame_count = 0
bootstrap_frames = int(2 * fps)  # use first 2 seconds for ID assignment
sim_threshold = 0.6


In [4]:
# Cell 5: Real-Time Tracking & Re-ID

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = []

    for box in results.boxes:
        if int(box.cls[0]) == 0:  # class 0 = player
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            detections.append([x1, y1, x2, y2])

    frame_tracks = []

    for bbox in detections:
        hist = extract_color_hist(frame, bbox)

        # During bootstrap: assign new IDs
        if frame_count < bootstrap_frames:
            player_db[next_id] = hist
            frame_tracks.append((bbox, next_id))
            next_id += 1

        # After bootstrap: match to existing
        else:
            best_id = None
            best_score = -1

            for player_id, stored_hist in player_db.items():
                sim = compute_similarity(hist, stored_hist)
                if sim > sim_threshold and sim > best_score:
                    best_score = sim
                    best_id = player_id

            if best_id is not None:
                frame_tracks.append((bbox, best_id))
            # Else: this is a new/unseen player; ignore (per rules)

    frame = draw_tracks(frame, frame_tracks)
    out.write(frame)
    frame_count += 1

    if frame_count % 10 == 0:
        print(f"Processed frame {frame_count}", end='\r')

cap.release()
out.release()
print(f"\n✅ Done. Output saved to: {output_path}")



0: 384x640 1 ball, 16 players, 2 referees, 52.3ms
Speed: 3.4ms preprocess, 52.3ms inference, 79.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 41.8ms
Speed: 1.2ms preprocess, 41.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 40.6ms
Speed: 1.1ms preprocess, 40.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 41.2ms
Speed: 1.0ms preprocess, 41.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 39.3ms
Speed: 1.0ms preprocess, 39.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 37.8ms
Speed: 2.0ms preprocess, 37.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 37.8ms
Speed: 1.0ms preprocess, 37.8ms inference, 1.0ms postprocess per image at sh

In [5]:
# Cell 6: Display output video in notebook
Video("reid_output1.mp4", embed=True)
